# 13 Interfaces, Protocols, and ABCs

## The Typing Map

![The Typing Map](./img/2023-10-04-08-21-17.png)

![Sequence ABCs](./img/2023-10-04-10-14-44.png)

## Two kinds of protocols

- Dynamic protocol: The informal protocols Python always had. Dynamic protocols are implicit, defined by convention, and described in the documentation. Python's most important dynamic protocols are supported by the interpreter itself, and are documented in the ["Data Model" chapter of The Python Language Reference](https://docs.python.org/3/reference/datamodel.html).
- Static protocol: A protocol as defined by [PEP 544--Protocols: Structural subtyping(static duck typing)](https://peps.python.org/pep-0544/), since Python 3.8. A static protocol has explicit definition: a `typing.Protocol` subclass

There are two key differences between them:
- An object may implement only part of dynamic protocol and still be useful; but to fulfill a static protocol, the object must provide every method declared in the protocol class, even if your program doesn't need them all.
- Static protocols can verified by static type checkers, but dynamic protocols can't.


In [6]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

In [7]:
v = Vowels()
v[0]

'A'

In [8]:
v[-1]

'U'

In [9]:
for c in v: print(c)

A
E
I
O
U


In [10]:
len(v)

TypeError: object of type 'Vowels' has no len()

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])
print(Card._fields)

ranks = [str(n) for n in range(2, 11)] + list('JQKA')
suits = 'spades diamonds clubs hearts'.split()
deck = [Card(rank, suit) for rank in ranks for suit in suits]
print(deck[:5])
print(len(deck))


('rank', 'suit')
[Card(rank='2', suit='spades'), Card(rank='2', suit='diamonds'), Card(rank='2', suit='clubs'), Card(rank='2', suit='hearts'), Card(rank='3', suit='spades')]
52


In [ ]:
from random import shuffle
shuffle(deck)
deck[:5]

[Card(rank='A', suit='hearts'),
 Card(rank='5', suit='diamonds'),
 Card(rank='4', suit='clubs'),
 Card(rank='2', suit='diamonds'),
 Card(rank='K', suit='diamonds')]

`str.isidentifier()` method explained

In [ ]:
print("hello".isidentifier())      # Output: True
print("123".isidentifier())        # Output: False (starts with a number)
print("_var".isidentifier())       # Output: True
print("if".isidentifier())         # Output: False (reserved keyword)
print("my_function".isidentifier())# Output: True
print("class".isidentifier())      # Output: False (reserved keyword)
print("some-var".isidentifier())   # Output: False (contains '-')


True
False
True
True
True
True
False


`namedtuple` fields must be identifiers.

In [ ]:
from collections import namedtuple

# Define a named tuple with valid field names
# '1pass' is not a valid field name
Person = namedtuple('Person', ['name', 'age', 'city', '1pass'])
p = Person('John', 25, 'New York', True)

print(p.name)  # Output: John
print(p.age)   # Output: 25
print(p.city)  # Output: New York

ValueError: Type names and field names must be valid identifiers: '1pass'

## Goose Typing

The Python Glossary entry for [abstract base class](https://docs.python.org/3/glossary.html#term-abstract-base-class) has a good explanation of the value they bring to duck-typed languages:
> Abstract base classes complement duck-typing by providing a way to define interfaces when other techniques like hasattr() would be clumsy or subtly wrong (for example with magic methods). ABCs introduce virtual subclasses, which are classes that don’t inherit from a class but are still recognized by isinstance() and issubclass(); see the abc module documentation. Python comes with many built-in ABCs for data structures (in the collections.abc module), numbers (in the numbers module), streams (in the io module), import finders and loaders (in the importlib.abc module). You can create your own ABCs with the abc module.

To summarize, goose typing entails:
- Subclassing from ABCs to make it explict that you are implementing a previously defined interface.
- Runtime type checking using ABCs instead of concrete classes as the second argument for `isinstance` and `issubclass`
- Don’t define custom ABCs (or metaclasses) in production code

## Subclassing an ABC

In [ ]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self) :
        self._cards = [Card(rank, suit) for rank in self.ranks
                                        for suit in self.suits]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)
        

## ABCs in the Standard Library

The most widely used ABCs are in `collections.abc`

![collections.abc](./img/2023-10-04-10-54-00.png)

- `Iterable` supports iteration with `__iter__`
- `Container` supports `in` and `not in` operators with `__contains__`
- `Sized` supports `len()` with `__len__`
- `Collection` is the subclass of `Iterable`, `Container`, and `Sized`, has no methods of its own
- `Sequence` supports indexing with `__getitem__`
- `MutableSequence` is a `Sequence` that can be modified
- `Mapping` supports `[]` with `__getitem__` 
- `MutableMapping` is a `Mapping` that can be modified
- `Set` supports `in` and `not in` operators with `__contains__` and `len()` with `__len__`
- `MutableSet` is a `Set` that can be modified
- `MappingView` objects returned from the mapping methods `keys()`, `items()`, and `values()` implement the interfaces `ItemsView`, `KeysView`, and `ValuesView`, respectively. The first two also implement the rich interface of `Set`
- `Iterator` subclasses `Iterable`, supports `next()` with `__next__`
- `Callable` supports `()` with `__call__`.  `Hashable` supports `hash()` with `__hash__`. These are not collections, but these two were deemed important enough to be included in `collections.abc`

## Defining and Using an ABC

> ABCs, like descriptors and metaclasses, are tools for building frameworks. Therefore, only a small minority of Python developers can create ABCs without imposing unreasonable limitations and needles work on fellow programmers.

![an ABC and three subclasses](./img/2023-10-04-14-35-12.png)

In [15]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""
        
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """
        
    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
                
        self.load(items)
        return tuple(items)

![Part of the Exception class hierarchy](./img/2023-10-04-14-51-39.png)

[Complete Python Exception hierarchy](https://docs.python.org/release/3.10.13/library/exceptions.html#exception-hierarchy)

In [ ]:
class Fake(Tombola):
    def pick(self):
        return 13

In [ ]:
Fake

__main__.Fake

In [ ]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

## ABC Syntax Details



In [ ]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):        
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        self.pick()
        

- Pretend we'll use this for online gaming. `random.SystemRandom` implements the `random` API on top of the `os.urandom(...)` function, which providesrandom bytes "suitable for cryptographic use", according to the [os model docs](https://docs.python.org/3/library/os.html#os.urandom).

In [ ]:
import random

class LottoBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(self._balls)

## A Virtual Subclass of an ABC

![TomboList](./img/2023-10-04-18-56-24.png)

In [41]:
from random import randrange

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
            
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(self)

- `TomboList` is registered as a virtual subclass of `Tombola`, `TomboList` doesn't need implement all abstract methods in `Tombola`, but we'll be caught usual runtime errors if these methods are called.
- `TomboList` extends list.
- `TomboList` inherits boolean behavior from `list`, and returns `True` if list is not empty.
- Our `pick` calls `self.pop`, inherited from `list`, passing a random index.
- `TomboList.load` is the same as `list.extend`.
- `loaded` delegates to `bool(self)`, which is inherited from `list`.
- It's always possible to call `register` in this way: `Tombola.register(TomboList)`, but it's only useful to do so when you need to register a class that you do not maintain, but which fulfill the interface.

In [31]:
issubclass(TomboList, Tombola)

True

In [42]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [34]:
issubclass(TomboList, list)

True

In [47]:
print(t.pick())
print(t.load([200]))

53
None


In [48]:
TomboList.__mro__

(__main__.TomboList, list, object)

## Structural Typing with ABCs


In [52]:
class Struggle:
    def __len__(self): return 23

from collections import abc
print(isinstance(Struggle(), abc.Sized))

# because abc.Sized implements __subclasshook__
print(issubclass(Struggle, abc.Sized))

True
True


The `__subclasshook__` for `Sized` checks whether the class argument has an attribute named `__len__`.

In [54]:
from abc import ABCMeta, abstractmethod

class Sized(metaclass=ABCMeta):
    
    __slots__ = ()
    
    @abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any('__len__' in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

## Static Protocols

### The Typed double Function

In [55]:
def double(x):
    return x * 2

print(double(1.5))
print(double([1, 2, 3]))
print(double("abc"))

from fractions import Fraction
print(double(Fraction(2, 5)))

3.0
[1, 2, 3, 1, 2, 3]
abcabc
4/5


In [76]:
from typing import TypeVar, Protocol

T = TypeVar('T') # 1

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: # 2
        ...
    
RT = TypeVar('RT', bound='Repeatable') # 3
    
def double(x: RT) -> RT: # 4
    return x * 2

1. We'll use this `T` in the `__mul__` signature.
2. `__mul__` is the essence of the `Repeatable` protocol. The `self` parameter is usually not annotated--its type is assumed to be the class. Here we use `T` to make sure the result type is the same as the type of `self`. Also, note that `repeat_count` is limited to `int` in this protocol.
3. The `RT` type variable is bounded by the `Repeatable` protocol: the type checker will require that the actual type implements `Repeatable`.
4. Now the type checker ia able to verify that the `x` parameter is an object that can be multiplied by an integer, and the return value has the same type as `x`.   

In [77]:
print(double(1.5))
print(double([1, 2, 3]))
print(double("abc"))

from fractions import Fraction
print(double(Fraction(2, 5)))

3.0
[1, 2, 3, 1, 2, 3]
abcabc
4/5


In [79]:
# Create a custom class that adheres to the Repeatable protocol
class MyRepeatableClass:
    def __init__(self, name: str):
        self.name = name

    def __mul__(self, repeat_count: int) -> 'MyRepeatableClass':
        return MyRepeatableClass(self.name * repeat_count)
        

# Create an object of your custom class
obj = MyRepeatableClass('Hello')

# Call the double function and pass the object as an argument
result = double(obj)

# Print the result
print(result.name)

HelloHello


### Runtime Checkable Static Protocols

The following protocols are provided by the typing module. All are decorated with `@runtime_checkable.`

- class `typing.SupportsAbs`: An ABC with one abstract method `__abs__` that is covariant in its return type.
- class `typing.SupportsBytes`: An ABC with one abstract method `__bytes__`.
- class `typing.SupportsComplex`: An ABC with one abstract method `__complex__`.
- class `typing.SupportsFloat`: An ABC with one abstract method _`_float__`.
- class `typing.SupportsIndex`: An ABC with one abstract method `__index__`.

New in version 3.8.
- class `typing.SupportsInt`: An ABC with one abstract method `__int__`.
- class `typing.SupportsRound`: An ABC with one abstract method `__round__` that is covariant in its return type.

In [83]:
from typing import SupportsComplex
import numpy as np

c64 = np.complex64(10 + 2j) # 1
print(c64)
print(isinstance(c64, complex)) # 2
print(isinstance(c64, SupportsComplex)) # 3
c = complex(c64) # 4
print(c)
print(isinstance(c, SupportsComplex)) # 5
print(complex(c))
print(dir(c))

(10+2j)
False
True
(10+2j)
False
(10+2j)
['__abs__', '__add__', '__bool__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__mul__', '__ne__', '__neg__', '__new__', '__pos__', '__pow__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__rpow__', '__rsub__', '__rtruediv__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', 'conjugate', 'imag', 'real']


1. `complex64` is one of the five complex number types provided by `NumPy`.
2. None of the NumPy complex types subclass the built-in `complex`.
3. But NumPy's complex types implements `__complex__`, so they comply with the `SupportsComplex` protocol.
4. Therefore, you can create built-in `complex` objects from NumPy complex numbers.
5. Sadly, the `complex` built-in type does not implement `__complex__`, although `complex(c)` works fine if `c` is a `complex`.

In [84]:
# if you want to test an object c is a complex or SupportsComplex, 
# you can provide a tuple of types as the second argument to isinstance
isinstance(c, (complex, SupportsComplex))

True

The built-in `complex` type and the NumPy `complex64` and `complex128` types are all registered as virtual subclasses of `numbers.Complex`.

The `numbers` ABCs are not recommended now, because they are not recognized by the static type checkers.

In [87]:
import numbers
print(isinstance(c, numbers.Complex))
print(isinstance(c64, numbers.Complex))

True
True


## Limitations of Runtime Protocol Checks

In [91]:
import sys
print(sys.version)
c =  3 + 4j
print(c.__float__)
print(c.__float__())

3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]


AttributeError: 'complex' object has no attribute '__float__'

In [94]:
from typing import SupportsFloat
print(isinstance(c, SupportsFloat))
print(issubclass(complex, SupportsFloat))

False
False


## Designing a Static Protocol

In [105]:
from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any:
        ...

In [96]:
import random
from typing import Any, Iterable, TYPE_CHECKING

class SimplePicker:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self) -> Any:
        return self._items.pop()
    
def test_isinstance() -> None:
    popper: RandomPicker = SimplePicker([1])
    assert isinstance(popper, RandomPicker)
    
def test_item_type() -> None:
    items = [1, 2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)
    assert isinstance(item, int)   

In [99]:
test_isinstance()
test_item_type()

In [106]:
import random
from typing import Any, Iterable, TYPE_CHECKING

class SimplePicker2:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

def test_isinstance2() -> None:
    popper: RandomPicker = SimplePicker2([1])
    # The protocol of pick method is not implemented by SimplePicker2
    # cause the AssertionError.
    assert isinstance(popper, RandomPicker)

test_isinstance2()

AssertionError: 

## The numbers ABCs and Numeric Protocols

In [116]:
from decimal import Decimal

# Decimal precision
decimal_num = Decimal('0.1000000000000000000000002') + Decimal('0.100000000000000000001') + Decimal('0.1000000000000000000000000003')
print(decimal_num)  # Output: 0.3

# Float precision
float_num = 0.1 + 0.1 + 0.1
print(float_num)  # Output: 0.30000000000000004

0.3000000000000000000010002003
0.30000000000000004


In [133]:
from decimal import Decimal, Context, MAX_PREC

print(MAX_PREC)
# Define the precision/scale
precision = 1

# Create a custom context with the desired precision
custom_context = Context(prec=precision)
decimal_num = Decimal('1.23456789000000000000000000000000000000111222333444', context=custom_context)
print(decimal_num) 

999999999999999999
1.23456789000000000000000000000000000000111222333444


The main takeaways for this section are:

- The `numbers` ABCs are fine for runtime type checking, but they are not recognized by the static type checkers.
- The numeric static protocols `SupportsComplex`, `SupportsFloat`, etc. work well for static typing, but are unreliable for runtime type checking when complex numbers are involved.